# Concurrency - Reader-Writer Lock

---

## Idea

A read–write lock (also called a shared–exclusive lock) is a concurrency control primitive that allows multiple threads to access a shared resource efficiently when those threads are only performing read operations. In many systems, reads are far more frequent than writes, so allowing parallel reading can greatly improve performance.  

The lock operates in two modes: **shared (read)** and **exclusive (write)**. When one or more threads acquire the lock in shared mode, they can safely read the data concurrently because none of them are mutating the underlying state. However, when a thread needs to modify that state, it must acquire the lock in exclusive mode. This prevents new readers from entering and blocks until all current readers have released their locks, ensuring that the writer has sole access.  

This mechanism strikes a balance between safety and throughput - maximizing concurrency during frequent reads while preserving strict consistency and atomicity during writes.

## Implementation

In [1]:
import logging
import threading
import time
from concurrent.futures import ThreadPoolExecutor
from contextlib import contextmanager

from theoria.validor import TestCase, Validor

In [2]:
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
if not log.hasHandlers():
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("[%(asctime)s] [%(levelname)s] %(message)s"))
    log.addHandler(handler)

In [3]:
class ReaderWriterLock:
    """
    A reader–writer lock allowing multiple readers or a single writer.

    Readers can acquire the lock concurrently, while a writer requires exclusive access.
    This implementation uses a condition variable to manage coordination between threads.

    It provides lightweight synchronization suitable for systems where reads are frequent
    and writes must remain consistent.
    """

    def __init__(self):
        self._lock = threading.Lock()
        self._cond = threading.Condition(self._lock)
        self._readers = 0
        self._writer = False
        self._waiting_writers = 0

    def acquire_read(self):
        with self._cond:
            # Block new readers if ANY writer is waiting
            while self._writer or self._waiting_writers > 0:
                self._cond.wait()
            self._readers += 1

    def release_read(self):
        with self._cond:
            self._readers -= 1
            if self._readers == 0:
                self._cond.notify_all()

    def acquire_write(self):
        with self._cond:
            self._waiting_writers += 1
            try:
                while self._writer or self._readers > 0:
                    self._cond.wait()
                self._writer = True
            finally:
                self._waiting_writers -= 1

    def release_write(self):
        with self._cond:
            self._writer = False
            self._cond.notify_all()
            
    @property
    def is_write_locked(self) -> bool:
        """Check if the lock is currently write-locked."""
        with self._lock:
            return self._writer

    # --- Context managers ---

    @contextmanager
    def read_lock(self):
        """Context manager for a read lock."""
        self.acquire_read()
        try:
            yield
        finally:
            self.release_read()

    @contextmanager
    def write_lock(self):
        """Context manager for a write lock."""
        self.acquire_write()
        try:
            yield
        finally:
            self.release_write()

## Tests

In [11]:
if __name__ == "__main__":

    def writer_exclusivity_test() -> bool:
        rwlock = ReaderWriterLock()
        shared = [0]
        writer_done = threading.Event()
        
        def writer():
            with rwlock.write_lock():
                shared[0] = 100
                
                # Hold lock for 1s
                writer_done.wait(timeout=1.0) 
        
        def reader(i: int) -> float:
            start = time.time()
            with rwlock.read_lock():
                elapsed = time.time() - start
                return elapsed
        
        # Start writer first
        w_thread = threading.Thread(target=writer)
        w_thread.start()
        time.sleep(0.05)  # Let writer acquire
        
        # Readers should block
        with ThreadPoolExecutor(max_workers=5) as exec:
            block_times = list(exec.map(reader, range(5)))

        w_thread.join()
        
        # All blocked > 0.5s
        return all(t > 0.5 for t in block_times)  

    read_tester = Validor(writer_exclusivity_test)
    read_tester.add_case(TestCase({}, True, "writer_exclusivity_test: blocks all readers"))
    read_tester.run()

[2026-01-03 19:38:57,298] [INFO] All 1 tests passed for writer_exclusivity_test.


In [12]:
if __name__ == "__main__":

    def test_readers_concurrent_not_serial() -> bool:
        rwlock = ReaderWriterLock()
        start_times = []
        end_times = []
        
        num_threads = 8
        work = 0.05  # seconds each reader holds the lock
        
        def reader(i: int):
            start = time.time()
            start_times.append(start)
            with rwlock.read_lock():
                time.sleep(work) 
            end_times.append(time.time())
        
        with ThreadPoolExecutor(max_workers=num_threads) as exec:
            exec.map(reader, range(num_threads))
        
        total_time = max(end_times) - min(start_times)
        
        # If readers ran serially, total_time would be >= num_threads * work
        return total_time < num_threads * work
    
    concurrent_tester = Validor(test_readers_concurrent_not_serial)
    concurrent_tester.add_case(TestCase({}, True, "test_readers_concurrent_not_serial: readers run concurrently"))
    concurrent_tester.run()

[2026-01-03 19:38:59,054] [INFO] All 1 tests passed for test_readers_concurrent_not_serial.


In [ ]:
if __name__ == "__main__":
    def test_readers_block_writer() -> bool:
        rwlock = ReaderWriterLock()
        readers_done = threading.Event()
        result = [0.0]
        
        wait_time = 1.0  # seconds each reader holds the lock
        
        def readers():
            with rwlock.read_lock():
                log.info("Reader acquired lock")
                readers_done.wait(timeout=wait_time)  # Hold lock
        
        def writer():
            start = time.time()
            with rwlock.write_lock():
                log.info("Writer acquired lock")
                result[0] = time.time() - start  # Measure BLOCKING time
        
        # Start readers
        reader_threads = [threading.Thread(target=readers) for _ in range(3)]
        for rt in reader_threads: 
            rt.start()
        
        time.sleep(0.1)  # Give readers time to pile up
        
        w_thread = threading.Thread(target=writer)
        w_thread.start()
        
        # Now release readers
        for rt in reader_threads: 
            rt.join()
        w_thread.join()
        
        block_time = result[0]

        return block_time > wait_time / 2 and block_time < 3.0
    
    writer_block_tester = Validor(test_readers_block_writer)
    writer_block_tester.add_case(TestCase({}, True, "readers block writer"))
    writer_block_tester.run()

[2026-01-03 19:40:20,005] [INFO] Reader acquired lock
[2026-01-03 19:40:20,005] [INFO] Reader acquired lock
[2026-01-03 19:40:20,008] [INFO] Reader acquired lock
[2026-01-03 19:40:21,010] [INFO] Writer acquired lock
[2026-01-03 19:40:21,012] [INFO] All 1 tests passed for test_readers_block_writer.


0.9014184474945068
